### Small simulation exercise

In our Econometrics course we recently had to solve the problem below in Stata - or specifically MATA. I don't really like MATA that much and decided to implement my solution in Python instead.

### Problem from the professor

In this exercise we conduct a small simulation study of both the finite-sample and asymptotic properties of Ordinary Least Squares (OLS). For this first problem, you should use MATA:
In particular, what you have learned in the assignment from last week. The data generating process we consider is given as:
$$
y_{i}=\alpha+\beta x_{i}+\varepsilon_{i}\tag{2}
$$
where
$$
\begin{aligned}
\alpha &=\beta=1 \\
x_{i} & \sim \mathcal{N}(0,1) \\
\varepsilon_{i} &=-x_{i-1}+z_{i} \\
z_{i} & \sim \mathcal{N}(0,1)
\end{aligned}
$$
Write a program that estimates (using OLS) and stores the slope coefficient of (2) for the following sample sizes $N=\{10,50,100,500,1000\},$ each based on $M=10000$ simulations from the true data generating process $(2) .$ Seed the random number generator such that results can be reproduced. 

__Based on your results, answer the following questions:__
* __a)__ _Is the OLS estimator unbiased?_
* __b)__ _Is it consistent?_
* __c)__ _Is it normally-distributed?_

In [1]:
import numpy as np
import pandas as pd

In [2]:
# information from assigment
samples = [10, 50, 100, 500, 1000]
simulations = 10000
# simulations = 10
alpha, beta = 1, 1

# seed random generator
np.random.seed(1)


def simulate(n):
    # randomly generated data...
    Z = np.random.normal(loc=0, scale=1, size=n)
    X = np.random.normal(loc=0, scale=1, size=n+1)
    
    X_lagged = X[:-1]  # all except last entry
    X_forward = X[1:]  # all except first entry
     
    epsilon = Z - X_lagged  # linear combination - generating error terms
    
    # generate Y
    Y = np.full(shape=n, fill_value=alpha) + beta * X_forward + epsilon
    
    # X matrix for OLS
    X_reg = np.vstack((np.ones(n), X_forward)).T
    
    # fit OLS model - normal form linear algebra
    betas = np.dot(np.dot(np.linalg.inv(np.dot(X_reg.transpose(), X_reg)), X_reg.transpose()), Y)

    return betas

In [3]:
# placeholder for simulation data
data_store = {value: np.ones((1,2)) for value in samples}

# run simulation
for sample in samples:
    
    for sim in range(simulations):
        s = simulate(sample)
        
        data_store[sample] = np.vstack((data_store[sample], s))
        
    data_store[sample] = data_store[sample][1:]

data_store

{10: array([[0.99931126, 0.82826271],
        [1.1307631 , 0.29289849],
        [1.26574025, 0.58624503],
        ...,
        [0.38469763, 2.03340239],
        [1.1733397 , 0.0065571 ],
        [1.32744505, 0.52918282]]),
 50: array([[1.05357392, 0.92043465],
        [1.52680437, 1.23043151],
        [0.83337842, 0.845356  ],
        ...,
        [0.71758886, 1.17936199],
        [1.08627917, 0.82793569],
        [1.2327566 , 1.43552453]]),
 100: array([[0.97379924, 1.05748841],
        [1.13790104, 0.9741253 ],
        [1.08969716, 0.90650596],
        ...,
        [0.76016251, 0.88240041],
        [0.8952219 , 1.30034966],
        [1.01711743, 0.98797412]]),
 500: array([[0.95794591, 0.97713657],
        [0.9420313 , 0.9475905 ],
        [0.929417  , 0.93598859],
        ...,
        [1.01455156, 1.03290912],
        [1.11007241, 1.04303472],
        [0.97007514, 0.99256359]]),
 1000: array([[1.07452571, 1.05321365],
        [1.00019313, 0.98979575],
        [0.95822183, 1.0482523 ]

In [4]:
# calculate descriptive statistics for data
def metrics(array):
    return {
        'mu': np.mean(array),
        'sigma_sq': np.var(array),
        'sigma': np.std(array)
    }
    
# make pretty output
results = {}

for key in data_store.keys():
    alpha_arr = data_store[key][:,0]
    beta_arr = data_store[key][:,1]
    
    alpha_metrics = metrics(alpha_arr)
    beta_metrics = metrics(beta_arr)
    
    results.update({
        (f'n = {key}', 'alpha'): alpha_metrics,
        (f'n = {key}', 'beta'): beta_metrics
    })
    
pd.DataFrame(results)

n = 10              n = 50             n = 100            \
             alpha      beta     alpha      beta     alpha      beta   
mu        1.000845  1.098551  0.996583  1.016770  0.998487  1.008050   
sigma_sq  0.234481  0.228770  0.040787  0.040272  0.020776  0.020494   
sigma     0.484232  0.478299  0.201957  0.200679  0.144138  0.143157   

           n = 500            n = 1000            
             alpha      beta     alpha      beta  
mu        1.000963  1.003647  0.999702  1.001197  
sigma_sq  0.004014  0.003997  0.001993  0.001989  
sigma     0.063356  0.063218  0.044647  0.044603

### Conclusion
We can answer the questions __a)__ through __c__ using the metrics calculated above.

This was a very unfun exercise to do in MATA but a bit more interesting in Python _(it also ran somewhat faster in python when using NumPy arrays)_.